### Capstone 2- Bengali Grapheme Classification Project

This workbook follows on the first experiments workbook. We take the best performing 25 models and check if image augmentation improves performance at all

In [1]:
#**********************************************************
#standard and ml imports
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
from collections import defaultdict
import mlflow
import random
from keras import backend as K
K.tensorflow_backend._get_available_gpus() # ensuring keras uses GPU (rocket emoji)

#**********************************************************
# using custom scripts for creating model and loading data
import sys
sys.path.insert(1, './model/')

# helper function to automate model building
from model_creator import model_create
# helper function to import data
from data_loader import data_loader
# helper function to do image threshold filtering and resizing
from image_processor import image_processor_func
# helper function to start and track mlflow experiments
from experiments import run_experiment
# function to output final predictions on submission set
from tester import test_func
# function to do data augmentation for multiple output convnets - from this kernel 
# https://www.kaggle.com/kaushal2896/bengali-graphemes-starter-eda-multi-output-cnn
from batch_generator import MultiOutputDataGenerator
# helper function to automate model building
from run_checker import results_accumulator

Using TensorFlow backend.


(106, 6)


In [2]:
# defining function to reset graph and set seeds - from Aurelien Gueron's book HandsOn ML
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
reset_graph()

Loading the results dataframe from the first set of experiments

In [5]:
#loading results dataframe
results = results_accumulator()
results.head()

,output_vowel_accuracy,output_consonant_accuracy,output_root_accuracy,val_output_consonant_accuracy,val_output_root_accuracy,val_output_vowel_accuracy
experiment_number,,,,,,
78,0.98189825,0.9843546,0.9373963,0.9597689509391785,0.8691495656967163,0.9577773213386536
48,0.9610525,0.9673372,0.8749474,0.9543915390968323,0.8488348722457886,0.955984890460968
2,0.97129834,0.9736219,0.91139436,0.9512049555778503,0.8546106219291687,0.9611631035804749
90,0.95377195,0.96193767,0.8194472,0.9607647657394409,0.8131846189498901,0.9531965851783752
67,0.85644734,0.8916993,0.5332271,0.9324835538864136,0.6767576336860657,0.9173471331596375


In [6]:
# sorting models by accuracy of root predictions on the validation set
selected_models = list(results.sort_values(by='val_output_root_accuracy',ascending=False).head(25).index.astype('int'))

Next, we read the project csv files which define the labels

In [10]:
# read the csv files
filenames = ['train','test','class_map','class_map_corrected','train_multi_diacritics','sample_submission']
df_dict = defaultdict()

for file in filenames:
    df_dict[file]=pd.read_csv('./data/{}.csv'.format(file))

Now, we start building our neural networks, start training and track training results using the MLFlow API. For this project, we use the MLFlow Keras Autologger

In [11]:
BATCH_SIZE = 100
EPOCHS = 50

In [12]:
# starting our experiments
import mlflow.keras
modelerrCount=0
experrCount=0
for index, experiment in experiment_matrix.iterrows():
    if index in selected_models:
        # resetting computational graph
        reset_graph()
        # specifying title
        title = 'exp_{0}_aug_yes_kernel_size_{1}_pool_size_{2}_dropout_rate_{3}_conv_layers_{4}_conv_filters_{5}'.format(
        index,int(experiment.loc['kernel_size']),int(experiment.loc['pool_size']),experiment.loc['dropout'],int(experiment.loc['conv_layers']),
        int(experiment.loc['conv_filters']))
        try:
            print('Creating Model')
            # create model with early stopping enabled
            model,callbacks_list = model_create(input_shape=(68,118,1),conv_kernel_size=int(experiment.loc['kernel_size']),
                                 pool_size=int(experiment.loc['pool_size']),
                                dropout_rate1=experiment.loc['dropout'],
                                 dropout_rate2=experiment.loc['dropout'],
                                n_conv_layers=int(experiment.loc['conv_layers']),
                                 conv_nfilters=int(experiment.loc['conv_filters']))
        except:
            # logging errors if any
            errCount+=1
            if errCount==1:
                with open('modelfailurelog.txt','w') as f:
                    f.write('Experiment {} failed - Check Model Setup'.format(title))
            else:
                with open('modelfailurelog.txt','a') as f:
                    f.write('Experiment {} failed - Check Model Setup'.format(title))
        print('Model Created')
        print(model.summary())
        print('Running Experiment-{}'.format(index))
        print("-------------------------------------")
        try:
            # running actual experiment
            run_experiment(title,model,callbacks_list,batch_size=BATCH_SIZE,epochs=EPOCHS,df_dict=df_dict,data_augmentation=True)
        except:
            # logging errors, if any
            experrCount+=1
            if experrCount==1:
                with open('experimentfailurelog.txt','w') as f:
                    f.write('Experiment {} failed - Check Exp Setup'.format(title))
            else:
                with open('experimentfailurelog.txt','a') as f:
                    f.write('Experiment {} failed - Check Exp Setup'.format(title))
print('All experiments finished')

Creating Model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model Created
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 68, 118, 1)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 68, 118, 50)  1850        input_layer[0][0]                
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 34, 59, 50)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 34, 59, 50)   90050       max_pooling2d_1

/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/mlflow/utils/autologging_utils.py:60: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  all_param_names, _, _, all_default_values = inspect.getargspec(fn)  # pylint: disable=W1505



Epoch 1/50
451/451 [==============================] - 32s 72ms/step - loss: 6.9591 - output_root_loss: 4.6819 - output_vowel_loss: 1.2314 - output_consonant_loss: 1.0458 - output_root_accuracy: 0.0429 - output_vowel_accuracy: 0.5746 - output_consonant_accuracy: 0.6439 - val_loss: 5.4918 - val_output_root_loss: 4.0008 - val_output_vowel_loss: 0.8079 - val_output_consonant_loss: 0.6648 - val_output_root_accuracy: 0.0902 - val_output_vowel_accuracy: 0.6997 - val_output_consonant_accuracy: 0.7495
Epoch 2/50
451/451 [==============================] - 26s 57ms/step - loss: 4.4982 - output_root_loss: 3.2994 - output_vowel_loss: 0.6158 - output_consonant_loss: 0.5829 - output_root_accuracy: 0.1772 - output_vowel_accuracy: 0.7883 - output_consonant_accuracy: 0.7932 - val_loss: 3.0625 - val_output_root_loss: 2.2603 - val_output_vowel_loss: 0.4036 - val_output_consonant_loss: 0.3914 - val_output_root_accuracy: 0.3732 - val_output_vowel_accuracy: 0.8630 - val_output_consonant_accuracy: 0.8648
Epo

/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/mlflow/utils/autologging_utils.py:60: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  all_param_names, _, _, all_default_values = inspect.getargspec(fn)  # pylint: disable=W1505


451/451 [==============================] - 25s 56ms/step - loss: 1.7800 - output_root_loss: 1.1434 - output_vowel_loss: 0.3310 - output_consonant_loss: 0.3055 - output_root_accuracy: 0.6661 - output_vowel_accuracy: 0.8885 - output_consonant_accuracy: 0.8977 - val_loss: 1.2605 - val_output_root_loss: 0.8733 - val_output_vowel_loss: 0.1867 - val_output_consonant_loss: 0.1959 - val_output_root_accuracy: 0.7397 - val_output_vowel_accuracy: 0.9422 - val_output_consonant_accuracy: 0.9391
Epoch 2/50
451/451 [==============================] - 25s 55ms/step - loss: 1.6226 - output_root_loss: 1.0268 - output_vowel_loss: 0.3085 - output_consonant_loss: 0.2872 - output_root_accuracy: 0.6948 - output_vowel_accuracy: 0.8996 - output_consonant_accuracy: 0.9020 - val_loss: 1.2594 - val_output_root_loss: 0.8574 - val_output_vowel_loss: 0.2072 - val_output_consonant_loss: 0.1883 - val_output_root_accuracy: 0.7594 - val_output_vowel_accuracy: 0.9343 - val_output_consonant_accuracy: 0.9357
Epoch 3/50
451/

...

451/451 [==============================] - 31s 68ms/step - loss: 2.8259 - output_root_loss: 1.9274 - output_vowel_loss: 0.4936 - output_consonant_loss: 0.4048 - output_root_accuracy: 0.4444 - output_vowel_accuracy: 0.8287 - output_consonant_accuracy: 0.8646 - val_loss: 1.7078 - val_output_root_loss: 1.2150 - val_output_vowel_loss: 0.2762 - val_output_consonant_loss: 0.2181 - val_output_root_accuracy: 0.6355 - val_output_vowel_accuracy: 0.9064 - val_output_consonant_accuracy: 0.9313
Epoch 2/50
451/451 [==============================] - 30s 67ms/step - loss: 2.5422 - output_root_loss: 1.7123 - output_vowel_loss: 0.4519 - output_consonant_loss: 0.3780 - output_root_accuracy: 0.4989 - output_vowel_accuracy: 0.8447 - output_consonant_accuracy: 0.8736 - val_loss: 1.5589 - val_output_root_loss: 1.1026 - val_output_vowel_loss: 0.2429 - val_output_consonant_loss: 0.2140 - val_output_root_accuracy: 0.6650 - val_output_vowel_accuracy: 0.9162 - val_output_consonant_accuracy: 0.9321
Epoch 3/50
451/

/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/mlflow/utils/autologging_utils.py:60: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  all_param_names, _, _, all_default_values = inspect.getargspec(fn)  # pylint: disable=W1505


451/451 [==============================] - 31s 68ms/step - loss: 1.9521 - output_root_loss: 1.2644 - output_vowel_loss: 0.3707 - output_consonant_loss: 0.3170 - output_root_accuracy: 0.6324 - output_vowel_accuracy: 0.8760 - output_consonant_accuracy: 0.8958 - val_loss: 1.1272 - val_output_root_loss: 0.7704 - val_output_vowel_loss: 0.1923 - val_output_consonant_loss: 0.1621 - val_output_root_accuracy: 0.7622 - val_output_vowel_accuracy: 0.9406 - val_output_consonant_accuracy: 0.9430
Epoch 2/50
451/451 [==============================] - 31s 68ms/step - loss: 1.8500 - output_root_loss: 1.1870 - output_vowel_loss: 0.3568 - output_consonant_loss: 0.3061 - output_root_accuracy: 0.6525 - output_vowel_accuracy: 0.8807 - output_consonant_accuracy: 0.8973 - val_loss: 1.1716 - val_output_root_loss: 0.7872 - val_output_vowel_loss: 0.2067 - val_output_consonant_loss: 0.1791 - val_output_root_accuracy: 0.7658 - val_output_vowel_accuracy: 0.9345 - val_output_consonant_accuracy: 0.9408
Epoch 3/50
451/

/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/mlflow/utils/autologging_utils.py:60: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  all_param_names, _, _, all_default_values = inspect.getargspec(fn)  # pylint: disable=W1505


451/451 [==============================] - 31s 68ms/step - loss: 1.6430 - output_root_loss: 1.0481 - output_vowel_loss: 0.3180 - output_consonant_loss: 0.2769 - output_root_accuracy: 0.6946 - output_vowel_accuracy: 0.8958 - output_consonant_accuracy: 0.9088 - val_loss: 0.9594 - val_output_root_loss: 0.6415 - val_output_vowel_loss: 0.1637 - val_output_consonant_loss: 0.1564 - val_output_root_accuracy: 0.8040 - val_output_vowel_accuracy: 0.9482 - val_output_consonant_accuracy: 0.9532
Epoch 2/50
451/451 [==============================] - 30s 68ms/step - loss: 1.5637 - output_root_loss: 0.9861 - output_vowel_loss: 0.3096 - output_consonant_loss: 0.2680 - output_root_accuracy: 0.7098 - output_vowel_accuracy: 0.8983 - output_consonant_accuracy: 0.9119 - val_loss: 0.9449 - val_output_root_loss: 0.5999 - val_output_vowel_loss: 0.1924 - val_output_consonant_loss: 0.1525 - val_output_root_accuracy: 0.8174 - val_output_vowel_accuracy: 0.9420 - val_output_consonant_accuracy: 0.9532
Epoch 3/50
451/